# Lab 3: Asking a Statistical Question

## William Olsen, partners with Jennifer Vo

Within this report we will be focusing on how to ask a statistical question.  Asking and answering a statistical question is generally requires the following three steps:

1) Writing down in words _precisely_ what question we are trying to ask.
2) Translating the precise english into a mathematical expression.  Often containing the PDF of the background - determining of which can be considered a substep of this step.  Then evaltating the integral.
3) Converting the probability from the previous step into a sigma.

In this lab we will presume that we know the background distribution and will focus on asking the statistical question and getting the correct results in a clear manor.

## Problem 1

